!pip install --upgrade py4j


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, IntegerType
from pyspark.ml.feature import Tokenizer, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import psycopg2

# Paramètres de connexion à PostgreSQL
db_host = "some-postgres"
db_port = "5432"
db_name = "mastodon_data"
db_user = "postgres"
db_password = "mysecretpassword"
db_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}"
db_properties = {
    "user": db_user,
    "password": db_password,
    "driver": "org.postgresql.Driver"
}

# Initialise SparkSession avec des ajustements de mémoire
spark = SparkSession.builder.appName("TootsSentimentAnalysis") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.2.25") \
    .master("local[*]") \
    .getOrCreate()

# Crée les tables si elles n'existent pas
def create_tables():
    conn = psycopg2.connect(
        host=db_host, port=db_port, dbname=db_name, user=db_user, password=db_password)
    cursor = conn.cursor()

    create_history_table = """
    CREATE TABLE IF NOT EXISTS history_toots (
        toot_id VARCHAR(255) PRIMARY KEY,
        timestamp TIMESTAMP,
        text TEXT,
        user_id VARCHAR(255),
        language VARCHAR(10),
        hashtags TEXT,
        reblogs_count INTEGER,
        favourites_count INTEGER,
        replies_count INTEGER
    );
    """

    create_sentiments_table = """
    CREATE TABLE IF NOT EXISTS toots_sentiments (
        toot_id VARCHAR(255) PRIMARY KEY,
        text TEXT,
        prediction DOUBLE PRECISION,
        sentiment VARCHAR(20)
    );
    """

    cursor.execute(create_history_table)
    cursor.execute(create_sentiments_table)
    conn.commit()
    cursor.close()
    conn.close()

    print("Tables créées ou déjà existantes.")

# Charge les données historiques depuis la table history_toots
def load_historical_data():
    print("Charge les données historiques depuis PostgreSQL (history_toots)...")
    try:
        historical_data = spark.read.jdbc(url=db_url, table="history_toots", properties=db_properties)
        print(f"Nombre de toots déjà traités (historique) : {historical_data.count()}")
        return historical_data
    except Exception as e:
        print(f"Erreur lors du chargement des données historiques : {e}")
        return None

# Fonction pour charger les données d'entraînement
def load_training_data():
    try:
        print("Chargement des données étiquetées pour l'entraînement...")
        data = spark.read.csv("tweets_dataset.csv", header=False, inferSchema=True)
        data = data.toDF("target", "ids", "date", "flag", "user", "text")
        data = data.withColumn("target", data["target"].cast(IntegerType()))
        data = data.withColumn("label", data["target"])
        data = data.na.drop(subset=["text", "label"])
        print(f"Nombre de lignes après suppression des valeurs nulles : {data.count()}")
        return data
    except Exception as e:
        print(f"Erreur lors du chargement des données d'entraînement : {e}")
        return None

# Fonction pour tokeniser et vectoriser les données
def preprocess_data(data):
    try:
        print("Tokenisation du texte...")
        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        tokenized_data = tokenizer.transform(data)

        print("Vectorisation du texte...")
        vectorizer = CountVectorizer(inputCol="words", outputCol="features")
        vectorizer_model = vectorizer.fit(tokenized_data)
        vectorized_data = vectorizer_model.transform(tokenized_data)

        return vectorized_data, vectorizer_model
    except Exception as e:
        print(f"Erreur lors du prétraitement des données : {e}")
        return None, None

# Fonction pour entraîner le modèle
def train_logistic_regression(vectorized_data):
    try:
        print("Division des données en ensembles d'entraînement et de test...")
        (training_data, test_data) = vectorized_data.sample(fraction=0.1, seed=42).randomSplit([0.8, 0.2], seed=42)
        print(f"Nombre de lignes d'entraînement : {training_data.count()}")
        print(f"Nombre de lignes de test : {test_data.count()}")

        print("Entraînement du modèle de régression logistique...")
        lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)
        lr_model = lr.fit(training_data)
        print("Modèle de régression logistique entraîné.")
        return lr_model, test_data
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle : {e}")
        return None, None

# Fonction pour évaluer le modèle
def evaluate_model(lr_model, test_data):
    try:
        print("Prédiction sur les données de test...")
        predictions = lr_model.transform(test_data)
        print("Aperçu des prédictions :")
        predictions.select("text", "label", "prediction").show(5)

        print("Évaluation de la précision du modèle...")
        evaluator = MulticlassClassificationEvaluator(
            labelCol='label', predictionCol='prediction', metricName='accuracy')
        accuracy = evaluator.evaluate(predictions)
        print(f"Précision du modèle : {accuracy}")
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle : {e}")


# Charge les nouveaux toots qui ne sont pas dans history_toots
def load_new_toots(historical_data):
    print("Charge les nouveaux toots depuis PostgreSQL (filtered_toots)...")
    try:
        all_toots = spark.read.jdbc(url=db_url, table="filtered_toots", properties=db_properties)
        new_toots = all_toots.join(historical_data, on="toot_id", how="left_anti")  
        print(f"Nombre de nouveaux toots : {new_toots.count()}")
        return new_toots
    except Exception as e:
        print(f"Erreur lors du chargement des nouveaux toots : {e}")
        return None

# Prédit les sentiments sur les nouveaux toots
def predict_sentiments(lr_model, vectorizer_model, new_toots):
    try:
        print("Tokenise les nouveaux toots...")
        tokenizer = Tokenizer(inputCol="text", outputCol="words")
        tokenized_toots = tokenizer.transform(new_toots)

        print("Vectorise les nouveaux toots...")
        vectorized_toots = vectorizer_model.transform(tokenized_toots)

        print("Prédit les sentiments sur les nouveaux toots...")
        sentiment_predictions = lr_model.transform(vectorized_toots)
        results = sentiment_predictions.select("toot_id", "text", "prediction")

        # Mappe les prédictions à des labels de sentiment
        print("Mappe les prédictions à des labels de sentiment...")
        def sentiment_label(prediction):
            if prediction == 0.0:
                return "négatif"
            elif prediction == 2.0:
                return "neutre"
            elif prediction == 4.0:
                return "positif"
            else:
                return "inconnu"

        sentiment_udf = udf(sentiment_label, StringType())
        results = results.withColumn("sentiment", sentiment_udf(results.prediction))
        print("Aperçu des résultats avec les labels de sentiment :")
        results.show(5)

        return results
    except Exception as e:
        print(f"Erreur lors de la prédiction des sentiments : {e}")
        return None

# Stocke les résultats dans toots_sentiments et déplace les toots dans history_toots
def store_results_in_postgres(results):
    print("Stocke les résultats dans PostgreSQL (toots_sentiments)...")
    try:
        results.write.jdbc(url=db_url, table="toots_sentiments", mode="append", properties=db_properties)
        print("Résultats stockés avec succès.")
    except Exception as e:
        print(f"Erreur lors du stockage des résultats : {e}")

# Déplace les toots traités dans history_toots
def move_toots_to_history(new_toots):
    print("Déplace les toots traités dans PostgreSQL (history_toots)...")
    try:
        new_toots.select("toot_id", "timestamp", "text", "user_id", "language", "hashtags", "reblogs_count", "favourites_count", "replies_count").write.jdbc(url=db_url, table="history_toots", mode="append", properties=db_properties)
        print("Toots déplacés dans la table historique avec succès.")
    except Exception as e:
        print(f"Erreur lors du déplacement des toots vers l'historique : {e}")

# Supprime les toots traités de filtered_toots
def delete_processed_toots(new_toots):
    print("Supprime les toots traités de la table filtered_toots...")
    conn = psycopg2.connect(
        host=db_host, port=db_port, dbname=db_name, user=db_user, password=db_password)
    cursor = conn.cursor()
    try:
        toot_ids = [row.toot_id for row in new_toots.select("toot_id").collect()]
        if toot_ids:
            formatted_ids = ','.join([f"'{toot_id}'" for toot_id in toot_ids])
            query = f"DELETE FROM filtered_toots WHERE toot_id IN ({formatted_ids})"
            print(f"Exécution de la requête : {query}")  
            cursor.execute(query)
            conn.commit()
            print(f"Toots supprimés de filtered_toots : {len(toot_ids)}")
        else:
            print("Aucun toot_id à supprimer.")
    except Exception as e:
        print(f"Erreur lors de la suppression des toots traités : {e}")
    finally:
        cursor.close()
        conn.close()


# Pipeline principal
def main():
    # Crée les tables si elles n'existent pas
    create_tables()

    # Charge les toots déjà traités (historique)
    historical_data = load_historical_data()
    if historical_data is None:
        return

    # Charge les nouveaux toots qui ne sont pas dans history_toots
    new_toots = load_new_toots(historical_data)
    if new_toots is None or new_toots.count() == 0:
        print("Aucun nouveau toot à traiter.")
        return

    # Charge les données d'entraînement et entraîne le modèle
    data = load_training_data() 
    if data is None:
        return

    vectorized_data, vectorizer_model = preprocess_data(data)
    if vectorized_data is None:
        return

    lr_model, test_data = train_logistic_regression(vectorized_data)
    if lr_model is not None:
        evaluate_model(lr_model, test_data)

        # Prédit les sentiments sur les nouveaux toots
        results = predict_sentiments(lr_model, vectorizer_model, new_toots)
        if results is not None:
            store_results_in_postgres(results)
            move_toots_to_history(new_toots)  
            delete_processed_toots(new_toots)  

    # Arrête SparkSession
    print("Arrêt de SparkSession...")
    spark.stop()
    print("SparkSession arrêtée.")

# Exécute le pipeline principal
if __name__ == "__main__":
    main()


Tables créées ou déjà existantes.
Charge les données historiques depuis PostgreSQL (history_toots)...
Nombre de toots déjà traités (historique) : 0
Charge les nouveaux toots depuis PostgreSQL (filtered_toots)...
Nombre de nouveaux toots : 2
Chargement des données étiquetées pour l'entraînement...
Nombre de lignes après suppression des valeurs nulles : 1600000
Tokenisation du texte...
Vectorisation du texte...
Division des données en ensembles d'entraînement et de test...
Nombre de lignes d'entraînement : 128069
Nombre de lignes de test : 31671
Entraînement du modèle de régression logistique...
Modèle de régression logistique entraîné.
Prédiction sur les données de test...
Aperçu des prédictions :
+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|@LettyA ahh ive a...|    0|       0.0|
|Damm back to scho...|    0|       0.0|
|wonders why someo...|    0|       0.0|
|Emily will be gla...|    0|       0.0|
|@machineplay 